# 🧬 Geometry-Complete Equivariant Diffusion
## De Novo Drug Design - **Kaggle P100**

**Prerequisites:** Upload `crossdocked_essential.tar.gz` as a Kaggle Dataset

## Cell 1: Setup

In [ ]:
import torch
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'
print(f'GPU: {gpu_name}')

!pip install -q torch-geometric rdkit scipy pyyaml tqdm scikit-learn
print('✅ Setup complete')

## Cell 2: Clone Repo

In [ ]:
import os

REPO = '/kaggle/working/geom_diffusion'
if not os.path.exists(REPO):
    !git clone https://github.com/Nethrananda21/geom_diffusion.git {REPO}
%cd {REPO}
!git pull origin master

## Cell 3: Extract Data

In [ ]:
import os

DATA_DIR = '/kaggle/working/data/raw'
os.makedirs(DATA_DIR, exist_ok=True)

# Check for Kaggle dataset input
kaggle_inputs = !ls /kaggle/input/
print(f'📁 Available inputs: {kaggle_inputs}')

# Find and extract the tar.gz
tar_found = False
for input_dir in os.listdir('/kaggle/input/'):
    tar_path = f'/kaggle/input/{input_dir}/crossdocked_essential.tar.gz'
    if os.path.exists(tar_path):
        print(f'📦 Extracting from {tar_path}...')
        !tar -xzf {tar_path} -C {DATA_DIR}/
        tar_found = True
        break

if not tar_found:
    print('❌ crossdocked_essential.tar.gz not found!')
    print('   Upload it as a Kaggle Dataset first.')
else:
    folders = [d for d in os.listdir(DATA_DIR) if os.path.isdir(f'{DATA_DIR}/{d}')]
    print(f'✅ Extracted {len(folders)} folders')

## Cell 4: Preprocess (P100: 2000 atoms)

In [ ]:
import os
import pickle
import numpy as np
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from rdkit import Chem
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# === P100 CONFIG ===
MAX_POCKET_ATOMS = 2000
MAX_LIGAND_ATOMS = 60
MIN_LIGAND_ATOMS = 5
MIN_LIGANDS_PER_POCKET = 3
N_POCKETS_PER_BIN = 60

ATOM_TYPES = ['C', 'N', 'O', 'S', 'P', 'F', 'Cl', 'Br', 'I', 'Other']
ATOM_TO_IDX = {a: i for i, a in enumerate(ATOM_TYPES)}

def parse_pdb(pdb_path):
    coords, types = [], []
    with open(pdb_path, 'r') as f:
        for line in f:
            if line.startswith('ATOM'):
                coords.append([float(line[30:38]), float(line[38:46]), float(line[46:54])])
                elem = line[76:78].strip() if len(line) > 77 else line[12:16].strip()[0]
                one_hot = [0.0] * 10
                one_hot[ATOM_TO_IDX.get(elem, 9)] = 1.0
                types.append(one_hot)
    return np.array(coords, dtype=np.float32), np.array(types, dtype=np.float32)

def parse_sdf(sdf_path):
    mol = Chem.SDMolSupplier(str(sdf_path), removeHs=True, sanitize=False)[0]
    if mol is None:
        return None, None
    try:
        conf = mol.GetConformer()
    except:
        return None, None
    coords, types = [], []
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        coords.append([pos.x, pos.y, pos.z])
        one_hot = [0.0] * 10
        one_hot[ATOM_TO_IDX.get(atom.GetSymbol(), 9)] = 1.0
        types.append(one_hot)
    return np.array(coords, dtype=np.float32), np.array(types, dtype=np.float32)

print(f'🔬 Preprocessing (max atoms: {MAX_POCKET_ATOMS})...')

DATA_DIR = '/kaggle/working/data/raw'
OUT_DIR = '/kaggle/working/data/crossdocked'
os.makedirs(OUT_DIR, exist_ok=True)

pocket_dirs = [Path(DATA_DIR) / d for d in os.listdir(DATA_DIR) if os.path.isdir(f'{DATA_DIR}/{d}')]
print(f'Found {len(pocket_dirs)} pocket directories')

pockets = defaultdict(list)
pocket_info = {}

for pocket_dir in tqdm(pocket_dirs, desc='Processing'):
    pocket_id = pocket_dir.name
    rec_pdbs = list(pocket_dir.glob('*_rec.pdb'))
    all_sdf = list(pocket_dir.glob('*.sdf'))
    
    if not rec_pdbs or not all_sdf:
        continue
    try:
        pocket_coords, pocket_types = parse_pdb(rec_pdbs[0])
        if len(pocket_coords) == 0 or len(pocket_coords) > MAX_POCKET_ATOMS:
            continue
    except:
        continue
    
    pocket_info[pocket_id] = {'size': len(pocket_coords)}
    
    for sdf_path in all_sdf[:50]:
        try:
            lig_coords, lig_types = parse_sdf(sdf_path)
            if lig_coords is None or len(lig_coords) < MIN_LIGAND_ATOMS or len(lig_coords) > MAX_LIGAND_ATOMS:
                continue
            pockets[pocket_id].append({
                'pocket_id': pocket_id,
                'ligand_id': sdf_path.stem,
                'ligand_coords': lig_coords,
                'ligand_types': lig_types,
                'pocket_coords': pocket_coords,
                'pocket_types': pocket_types
            })
        except:
            continue

print(f'\n✅ Processed {len(pockets)} valid pockets')

valid = [p for p, s in pockets.items() if len(s) >= MIN_LIGANDS_PER_POCKET]
print(f'After filter: {len(valid)} pockets')

if valid:
    sizes = [pocket_info[p]['size'] for p in valid]
    p33, p66 = np.percentile(sizes, [33, 66])
    small = [p for p in valid if pocket_info[p]['size'] <= p33]
    medium = [p for p in valid if p33 < pocket_info[p]['size'] <= p66]
    large = [p for p in valid if pocket_info[p]['size'] > p66]
    
    np.random.shuffle(small); np.random.shuffle(medium); np.random.shuffle(large)
    n_per = min(N_POCKETS_PER_BIN, len(small), len(medium), len(large))
    selected = small[:n_per] + medium[:n_per] + large[:n_per]
    
    np.random.shuffle(selected)
    split_idx = int(len(selected) * 0.85)
    train_pockets, val_pockets = selected[:split_idx], selected[split_idx:]
    
    train_samples = [s for p in train_pockets for s in pockets[p]]
    val_samples = [s for p in val_pockets for s in pockets[p]]
    
    print(f'📊 Train: {len(train_samples)}, Val: {len(val_samples)}')
    
    with open(f'{OUT_DIR}/train_data.pkl', 'wb') as f:
        pickle.dump(train_samples, f)
    with open(f'{OUT_DIR}/val_data.pkl', 'wb') as f:
        pickle.dump(val_samples, f)
    print('💾 Saved!')

## Cell 5: Update Config for Kaggle Paths

In [ ]:
import yaml

%cd /kaggle/working/geom_diffusion

with open('configs/p100.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Update paths for Kaggle
cfg['data']['root'] = '/kaggle/working/data/crossdocked'

with open('configs/p100.yaml', 'w') as f:
    yaml.dump(cfg, f)

print('✅ Config updated for Kaggle paths')

## Cell 6: Train 🚀

In [ ]:
import shutil
from pathlib import Path

%cd /kaggle/working/geom_diffusion

# Clear cache
if Path('data/cache').exists():
    shutil.rmtree('data/cache')

!python train.py --config configs/p100.yaml --checkpoint_dir /kaggle/working/checkpoints

## Cell 7: Monitor GPU

In [ ]:
!nvidia-smi

## Cell 8: Save Checkpoints to Output

In [ ]:
# Kaggle saves /kaggle/working/ as output
# Your checkpoints are already in /kaggle/working/checkpoints
!ls -lh /kaggle/working/checkpoints/